# Diodes and Semiconductor Models

This notebook covers diode modeling in Pulsim, from ideal diodes to detailed Shockley models.

## Contents
1. Ideal Diode Model
2. Shockley Diode Equation
3. Diode Parameters
4. Junction Capacitance
5. Rectifier Circuits
6. Diode Library

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

# Set up nice plots
plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 11

## 1. Ideal Diode Model

The simplest diode model: 
- Forward biased: Short circuit (0V drop)
- Reverse biased: Open circuit (no current)

```python
DiodeParams:
    ideal = True  # Use ideal model
```

In [ ]:
# Create an ideal diode
diode_ideal = pulsim.DiodeParams()
diode_ideal.ideal = True

print("Ideal Diode Parameters:")
print(f"  Model: {'Ideal (on/off)' if diode_ideal.ideal else 'Shockley'}")
print(f"  Forward: Short circuit")
print(f"  Reverse: Open circuit")

In [ ]:
# Half-wave rectifier with ideal diode
netlist = '''
{
  "name": "Ideal Diode Rectifier",
  "components": [
    {"type": "V", "name": "Vin", "nodes": ["in", "0"],
     "waveform": {"type": "sine", "offset": 0, "amplitude": 10, "frequency": 60}},
    {"type": "D", "name": "D1", "nodes": ["in", "out"], "params": {"ideal": true}},
    {"type": "R", "name": "Rload", "nodes": ["out", "0"], "value": 1000}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 50e-3  # 3 cycles at 60Hz
options.dt = 10e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3
v_in = np.array(data['signals']['V(in)'])
v_out = np.array(data['signals']['V(out)'])

plt.figure(figsize=(12, 5))
plt.plot(time_ms, v_in, 'b-', linewidth=1.5, label='V(in) - AC Input')
plt.plot(time_ms, v_out, 'r-', linewidth=1.5, label='V(out) - Rectified')
plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('Half-Wave Rectifier with Ideal Diode')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 2. Shockley Diode Equation

The Shockley equation models the exponential I-V characteristic:

$$I_D = I_S \left( e^{\frac{V_D}{n \cdot V_T}} - 1 \right)$$

Where:
- $I_S$ = Saturation current (typically 1e-14 A)
- $n$ = Ideality factor (1.0 to 2.0)
- $V_T$ = Thermal voltage ($kT/q \approx 26mV$ at 300K)

```python
DiodeParams:
    is_ = 1e-14   # Saturation current (A)
    n   = 1.0     # Ideality factor
    rs  = 0.0     # Series resistance (Ω)
    vt  = 0.026   # Thermal voltage (V)
    ideal = False # Use Shockley model
```

In [ ]:
# Plot Shockley diode I-V curves for different parameters
V = np.linspace(-0.5, 0.8, 1000)
VT = 0.026  # Thermal voltage

def shockley(V, Is, n):
    """Shockley diode equation."""
    return Is * (np.exp(V / (n * VT)) - 1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Effect of saturation current
for Is in [1e-14, 1e-12, 1e-10]:
    I = shockley(V, Is, n=1.0)
    I = np.clip(I, -1e-3, 100e-3)  # Limit for plotting
    axes[0].plot(V, I * 1e3, linewidth=2, label=f'Is = {Is:.0e} A')

axes[0].set_xlabel('Voltage (V)')
axes[0].set_ylabel('Current (mA)')
axes[0].set_title('Effect of Saturation Current (Is)')
axes[0].legend()
axes[0].grid(True)
axes[0].set_ylim([-5, 100])

# Effect of ideality factor
for n in [1.0, 1.5, 2.0]:
    I = shockley(V, Is=1e-14, n=n)
    I = np.clip(I, -1e-3, 100e-3)
    axes[1].plot(V, I * 1e3, linewidth=2, label=f'n = {n}')

axes[1].set_xlabel('Voltage (V)')
axes[1].set_ylabel('Current (mA)')
axes[1].set_title('Effect of Ideality Factor (n)')
axes[1].legend()
axes[1].grid(True)
axes[1].set_ylim([-5, 100])

plt.tight_layout()
plt.show()

In [ ]:
# Create Shockley diode model
diode_shockley = pulsim.DiodeParams()
diode_shockley.is_ = 1e-14  # Note: 'is_' because 'is' is Python keyword
diode_shockley.n = 1.0
diode_shockley.rs = 0.01    # 10mΩ series resistance
diode_shockley.vt = 0.026
diode_shockley.ideal = False

print("Shockley Diode Parameters:")
print(f"  Saturation current (Is): {diode_shockley.is_:.2e} A")
print(f"  Ideality factor (n): {diode_shockley.n}")
print(f"  Series resistance (Rs): {diode_shockley.rs*1000:.1f} mΩ")
print(f"  Thermal voltage (Vt): {diode_shockley.vt*1000:.0f} mV")

In [ ]:
# Compare ideal vs Shockley diode in rectifier
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

for idx, (model, label) in enumerate([(True, 'Ideal'), (False, 'Shockley')]):
    netlist = f'''
    {{
      "name": "{label} Diode Rectifier",
      "components": [
        {{"type": "V", "name": "Vin", "nodes": ["in", "0"],
         "waveform": {{"type": "sine", "offset": 0, "amplitude": 10, "frequency": 60}}}},
        {{"type": "D", "name": "D1", "nodes": ["in", "out"], 
         "params": {{"ideal": {str(model).lower()}, "is": 1e-14, "n": 1.0, "rs": 0.01}}}},
        {{"type": "R", "name": "Rload", "nodes": ["out", "0"], "value": 100}}
      ]
    }}
    '''
    
    circuit = pulsim.parse_netlist_string(netlist)
    options = pulsim.SimulationOptions()
    options.tstop = 33e-3  # 2 cycles
    options.dt = 10e-6
    
    result = pulsim.simulate(circuit, options)
    data = result.to_dict()
    
    time_ms = np.array(data['time']) * 1e3
    v_in = np.array(data['signals']['V(in)'])
    v_out = np.array(data['signals']['V(out)'])
    v_diode = v_in - v_out  # Voltage across diode
    
    axes[idx].plot(time_ms, v_in, 'b-', linewidth=1.5, alpha=0.5, label='V(in)')
    axes[idx].plot(time_ms, v_out, 'r-', linewidth=2, label='V(out)')
    axes[idx].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    axes[idx].set_ylabel('Voltage (V)')
    axes[idx].set_title(f'{label} Diode Model')
    axes[idx].legend(loc='upper right')
    axes[idx].grid(True)

axes[1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

print("Notice: Shockley model shows ~0.7V forward voltage drop")

## 3. Diode Parameters Deep Dive

Full set of diode parameters in Pulsim:

```python
DiodeParams:
    # Basic Shockley parameters
    is_ = 1e-14   # Saturation current
    n   = 1.0     # Ideality factor
    rs  = 0.0     # Series resistance
    vt  = 0.026   # Thermal voltage
    
    # Junction capacitance
    cj0 = 0.0     # Zero-bias junction capacitance (F)
    vj  = 0.7     # Junction potential (V)
    m   = 0.5     # Grading coefficient
    tt  = 0.0     # Transit time (s) - for diffusion capacitance
    
    # Breakdown
    bv  = 100.0   # Reverse breakdown voltage (V)
    ibv = 1e-10   # Current at breakdown (A)
```

In [ ]:
# Create a complete diode model (1N4007-like)
diode = pulsim.DiodeParams()

# Basic parameters
diode.is_ = 2.55e-9      # Higher Is for rectifier
diode.n = 1.75           # Typical for Si rectifier
diode.rs = 0.042         # Series resistance (42mΩ)
diode.vt = 0.026
diode.ideal = False

# Junction capacitance
diode.cj0 = 15e-12       # 15pF zero-bias capacitance
diode.vj = 0.7           # Junction potential
diode.m = 0.5            # Abrupt junction
diode.tt = 3e-6          # 3µs transit time (slow recovery)

# Breakdown
diode.bv = 1000.0        # 1000V breakdown (1N4007)
diode.ibv = 5e-6

print("Complete Diode Model (1N4007-like):")
print(f"\nDC Parameters:")
print(f"  Is = {diode.is_:.2e} A")
print(f"  n = {diode.n}")
print(f"  Rs = {diode.rs*1000:.0f} mΩ")
print(f"\nJunction Capacitance:")
print(f"  Cj0 = {diode.cj0*1e12:.0f} pF")
print(f"  Vj = {diode.vj} V")
print(f"  m = {diode.m}")
print(f"  tt = {diode.tt*1e6:.0f} µs")
print(f"\nBreakdown:")
print(f"  BV = {diode.bv} V")

## 4. Junction Capacitance

The junction capacitance varies with reverse voltage:

$$C_j(V) = \frac{C_{j0}}{\left(1 - \frac{V}{V_j}\right)^m}$$

Where:
- $C_{j0}$ = Zero-bias capacitance
- $V_j$ = Junction potential (~0.7V for Si)
- $m$ = Grading coefficient (0.33 for linear, 0.5 for abrupt)

In [ ]:
# Plot junction capacitance vs voltage
V_reverse = np.linspace(0, 100, 1000)
Cj0 = 15e-12  # 15pF
Vj = 0.7

def junction_capacitance(V, Cj0, Vj, m):
    """Calculate junction capacitance vs reverse voltage."""
    return Cj0 / (1 + V/Vj)**m

plt.figure(figsize=(10, 5))

for m, label in [(0.33, 'Linear graded (m=0.33)'), 
                  (0.5, 'Abrupt junction (m=0.5)')]:
    Cj = junction_capacitance(V_reverse, Cj0, Vj, m)
    plt.plot(V_reverse, Cj * 1e12, linewidth=2, label=label)

plt.xlabel('Reverse Voltage (V)')
plt.ylabel('Junction Capacitance (pF)')
plt.title('Diode Junction Capacitance vs Reverse Bias')
plt.legend()
plt.grid(True)
plt.xlim([0, 100])
plt.tight_layout()
plt.show()

print("Junction capacitance decreases with reverse bias.")
print("This is important for high-frequency switching applications.")

## 5. Rectifier Circuits

Let's simulate common rectifier topologies.

In [ ]:
# Full-wave bridge rectifier
netlist = '''
{
  "name": "Full-Wave Bridge Rectifier",
  "components": [
    {"type": "V", "name": "Vac", "nodes": ["ac_p", "ac_n"],
     "waveform": {"type": "sine", "offset": 0, "amplitude": 12, "frequency": 60}},
    
    {"type": "D", "name": "D1", "nodes": ["ac_p", "dc_p"], 
     "params": {"ideal": false, "is": 2.55e-9, "n": 1.75, "rs": 0.042}},
    {"type": "D", "name": "D2", "nodes": ["ac_n", "dc_p"], 
     "params": {"ideal": false, "is": 2.55e-9, "n": 1.75, "rs": 0.042}},
    {"type": "D", "name": "D3", "nodes": ["dc_n", "ac_p"], 
     "params": {"ideal": false, "is": 2.55e-9, "n": 1.75, "rs": 0.042}},
    {"type": "D", "name": "D4", "nodes": ["dc_n", "ac_n"], 
     "params": {"ideal": false, "is": 2.55e-9, "n": 1.75, "rs": 0.042}},
    
    {"type": "C", "name": "Cfilter", "nodes": ["dc_p", "dc_n"], "value": 1000e-6},
    {"type": "R", "name": "Rload", "nodes": ["dc_p", "dc_n"], "value": 100}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 100e-3  # 6 cycles
options.dt = 10e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3
v_ac = np.array(data['signals']['V(ac_p)']) - np.array(data['signals'].get('V(ac_n)', np.zeros_like(data['signals']['V(ac_p)'])))
v_dc = np.array(data['signals']['V(dc_p)']) - np.array(data['signals'].get('V(dc_n)', np.zeros_like(data['signals']['V(dc_p)'])))

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

axes[0].plot(time_ms, data['signals']['V(ac_p)'], 'b-', linewidth=1.5)
axes[0].set_ylabel('AC Input (V)')
axes[0].set_title('Full-Wave Bridge Rectifier with Filter Capacitor')
axes[0].grid(True)

axes[1].plot(time_ms, data['signals']['V(dc_p)'], 'r-', linewidth=2)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('DC Output (V)')
axes[1].grid(True)

# Calculate ripple
v_dc_arr = np.array(data['signals']['V(dc_p)'])
v_dc_ss = v_dc_arr[len(v_dc_arr)//2:]  # Steady state
ripple_pp = np.max(v_dc_ss) - np.min(v_dc_ss)
v_dc_avg = np.mean(v_dc_ss)

axes[1].axhline(y=v_dc_avg, color='g', linestyle='--', label=f'Avg: {v_dc_avg:.2f}V')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nRectifier Performance:")
print(f"  DC output voltage: {v_dc_avg:.2f} V")
print(f"  Peak-to-peak ripple: {ripple_pp*1e3:.1f} mV")
print(f"  Ripple percentage: {ripple_pp/v_dc_avg*100:.2f}%")

In [ ]:
# Effect of filter capacitor size
capacitor_values = [100e-6, 470e-6, 1000e-6, 2200e-6]

plt.figure(figsize=(12, 6))

for C in capacitor_values:
    netlist = f'''
    {{
      "name": "Rectifier C={C*1e6:.0f}uF",
      "components": [
        {{"type": "V", "name": "Vac", "nodes": ["ac_p", "0"],
         "waveform": {{"type": "sine", "offset": 0, "amplitude": 12, "frequency": 60}}}},
        {{"type": "D", "name": "D1", "nodes": ["ac_p", "dc_p"], 
         "params": {{"ideal": false, "is": 2.55e-9, "n": 1.75}}}},
        {{"type": "D", "name": "D2", "nodes": ["0", "dc_p"], 
         "params": {{"ideal": false, "is": 2.55e-9, "n": 1.75}}}},
        {{"type": "D", "name": "D3", "nodes": ["0", "ac_p"], 
         "params": {{"ideal": false, "is": 2.55e-9, "n": 1.75}}}},
        {{"type": "C", "name": "Cfilter", "nodes": ["dc_p", "0"], "value": {C}}},
        {{"type": "R", "name": "Rload", "nodes": ["dc_p", "0"], "value": 100}}
      ]
    }}
    '''
    
    circuit = pulsim.parse_netlist_string(netlist)
    options = pulsim.SimulationOptions()
    options.tstop = 100e-3
    options.dt = 10e-6
    
    result = pulsim.simulate(circuit, options)
    data = result.to_dict()
    
    time_ms = np.array(data['time']) * 1e3
    v_dc = np.array(data['signals']['V(dc_p)'])
    
    # Calculate ripple
    v_ss = v_dc[len(v_dc)//2:]
    ripple = np.max(v_ss) - np.min(v_ss)
    
    plt.plot(time_ms, v_dc, linewidth=1.5, 
             label=f'C = {C*1e6:.0f} µF (ripple: {ripple*1e3:.0f} mV)')

plt.xlabel('Time (ms)')
plt.ylabel('DC Output Voltage (V)')
plt.title('Effect of Filter Capacitor on Ripple')
plt.legend()
plt.grid(True)
plt.xlim([50, 100])
plt.tight_layout()
plt.show()

## 6. Diode Library

Pulsim includes pre-defined models for common diodes:

| Function | Diode | Type | Key Specs |
|----------|-------|------|----------|
| `diode_1N4007()` | 1N4007 | Rectifier | 1000V, 1A, slow |
| `diode_1N4148()` | 1N4148 | Signal | 100V, fast switching |
| `diode_1N5819()` | 1N5819 | Schottky | 40V, low Vf |
| `diode_MUR860()` | MUR860 | Fast Recovery | 600V, 50ns |
| `diode_C3D10065A()` | C3D10065A | SiC Schottky | 650V, zero Qrr |

In [ ]:
# Compare diode I-V characteristics from library
diodes = {
    '1N4007 (Rectifier)': {'is': 2.55e-9, 'n': 1.75, 'rs': 0.042},
    '1N4148 (Signal)': {'is': 2.52e-9, 'n': 1.752, 'rs': 0.568},
    '1N5819 (Schottky)': {'is': 1e-5, 'n': 1.1, 'rs': 0.04},
    'C3D10065A (SiC)': {'is': 1e-15, 'n': 1.05, 'rs': 0.065},
}

V = np.linspace(0, 1.5, 1000)
VT = 0.026

plt.figure(figsize=(12, 6))

for name, params in diodes.items():
    # Shockley + series resistance: V = Vd + I*Rs
    # Solve iteratively
    I = np.zeros_like(V)
    for i, Vtotal in enumerate(V):
        # Newton iteration
        Vd = Vtotal * 0.9  # Initial guess
        for _ in range(20):
            Id = params['is'] * (np.exp(Vd / (params['n'] * VT)) - 1)
            Vd_calc = Vtotal - Id * params['rs']
            if abs(Vd - Vd_calc) < 1e-9:
                break
            Vd = Vd_calc
        I[i] = max(0, Id)
    
    I = np.clip(I, 0, 5)  # Limit to 5A for plot
    plt.plot(V, I, linewidth=2, label=name)

plt.xlabel('Forward Voltage (V)')
plt.ylabel('Forward Current (A)')
plt.title('Diode I-V Characteristics from Library')
plt.legend()
plt.grid(True)
plt.xlim([0, 1.5])
plt.ylim([0, 3])
plt.tight_layout()
plt.show()

print("Key observations:")
print("  - Schottky (1N5819): Lowest forward voltage (~0.3V)")
print("  - SiC (C3D10065A): Higher Vf but better high-temp performance")
print("  - Signal (1N4148): Higher Rs, suitable for small signals")
print("  - Rectifier (1N4007): Good balance for power applications")

In [ ]:
# Using library diodes in simulation
# Compare Schottky vs standard diode in buck converter freewheeling

diode_types = [
    ('Standard (1N4007)', {'is': 2.55e-9, 'n': 1.75, 'rs': 0.042}),
    ('Schottky (1N5819)', {'is': 1e-5, 'n': 1.1, 'rs': 0.04}),
]

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

for diode_name, diode_params in diode_types:
    netlist = f'''
    {{
      "name": "Buck with {diode_name}",
      "components": [
        {{"type": "V", "name": "Vin", "nodes": ["vin", "0"], "value": 12}},
        {{"type": "V", "name": "Vpwm", "nodes": ["ctrl", "0"],
         "waveform": {{"type": "pwm", "v_off": 0, "v_on": 5, "frequency": 100e3, "duty": 0.5}}}},
        {{"type": "S", "name": "S1", "nodes": ["vin", "sw", "ctrl", "0"], 
         "params": {{"ron": 0.01, "roff": 1e9, "vth": 2.5}}}},
        {{"type": "D", "name": "D1", "nodes": ["0", "sw"], 
         "params": {{"ideal": false, "is": {diode_params['is']}, 
                    "n": {diode_params['n']}, "rs": {diode_params['rs']}}}}},
        {{"type": "L", "name": "L1", "nodes": ["sw", "vout"], "value": 100e-6}},
        {{"type": "C", "name": "C1", "nodes": ["vout", "0"], "value": 100e-6}},
        {{"type": "R", "name": "Rload", "nodes": ["vout", "0"], "value": 6}}
      ]
    }}
    '''
    
    circuit = pulsim.parse_netlist_string(netlist)
    options = pulsim.SimulationOptions()
    options.tstop = 200e-6
    options.dt = 10e-9
    
    result = pulsim.simulate(circuit, options)
    data = result.to_dict()
    
    time_us = np.array(data['time']) * 1e6
    v_sw = np.array(data['signals']['V(sw)'])
    
    # Plot switch node
    axes[0].plot(time_us, v_sw, linewidth=1, label=diode_name)

axes[0].set_ylabel('Switch Node (V)')
axes[0].set_title('Buck Converter Switch Node Voltage')
axes[0].legend()
axes[0].grid(True)

# Zoom on negative voltage (diode drop)
axes[1].plot(time_us, v_sw, linewidth=1)
axes[1].set_xlabel('Time (µs)')
axes[1].set_ylabel('Switch Node (V)')
axes[1].set_title('Diode Forward Voltage Drop (when low-side conducts)')
axes[1].set_ylim([-1.5, 0.5])
axes[1].grid(True)
axes[1].axhline(y=-0.7, color='r', linestyle='--', label='~0.7V (1N4007)')
axes[1].axhline(y=-0.3, color='g', linestyle='--', label='~0.3V (Schottky)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\nSchottky advantage: Lower forward voltage drop = lower conduction losses")

## Summary

### Diode Models in Pulsim

| Model | Use Case | Parameters |
|-------|----------|------------|
| **Ideal** | Quick analysis | `ideal=True` |
| **Shockley** | Accurate DC | `is_, n, rs, vt` |
| **Full** | Transient/AC | + `cj0, vj, m, tt, bv` |

### Key Parameters

- **Is**: Saturation current - affects forward voltage
- **n**: Ideality factor - slope of I-V curve
- **Rs**: Series resistance - loss at high currents
- **Cj0**: Junction capacitance - switching speed
- **tt**: Transit time - reverse recovery

### Library Diodes

- **1N4007**: General purpose rectifier
- **1N4148**: Fast signal diode
- **1N5819**: Schottky for low Vf
- **MUR860**: Fast recovery for power
- **C3D10065A**: SiC for high efficiency

**Next:** [MOSFETs and IGBTs](07_mosfets_igbts.ipynb)